# Week 2 — Demand by Line & Monthly Trends (Buenos Aires Subte)

**Objective:** Load the full 2024 turnstiles dataset (24 CSVs), clean/normalize,
compute demand by line and monthly trends, and generate first visuals
for the BI dashboard and the LinkedIn Week 2 update.

**Outputs:**
- `data/processed/agg_passengers_by_line_full.csv`
- `data/processed/trend_passengers_by_line_month_full.csv`


In [2]:
# === Week 2: Setup (minimal) ===
import os, glob, re, csv
import pandas as pd
import plotly.express as px  # para los gráficos de los bloques 6+

pd.options.display.max_rows = 10
pd.options.display.float_format = "{:,.0f}".format

# Paths
BASE_DIR = os.path.abspath("..")   # desde /notebooks
RAW_DIR  = os.path.join(BASE_DIR, "data", "raw")
PROC_DIR = os.path.join(BASE_DIR, "data", "processed")
MOL_DIR  = os.path.join(RAW_DIR, "molinetes")

os.makedirs(PROC_DIR, exist_ok=True)

print("RAW_DIR:", RAW_DIR)
print("PROC_DIR:", PROC_DIR)
print("MOL_DIR:", MOL_DIR, "exists?", os.path.exists(MOL_DIR))



RAW_DIR: c:\Users\do_ch\OneDrive\Escritorio\Proyectos\Proyectos GitHub\subte-dashboard\data\raw
PROC_DIR: c:\Users\do_ch\OneDrive\Escritorio\Proyectos\Proyectos GitHub\subte-dashboard\data\processed
MOL_DIR: c:\Users\do_ch\OneDrive\Escritorio\Proyectos\Proyectos GitHub\subte-dashboard\data\raw\molinetes exists? True


In [3]:
# === Week 2: Load & Clean Full 2024 Turnstiles (24 CSVs) ===
import os, glob, re, csv
import pandas as pd

pd.options.display.max_rows = 10
pd.options.display.float_format = '{:,.0f}'.format

# ---- Paths (self-contained) ----
BASE_DIR = os.path.abspath("..")          # from /notebooks
RAW_DIR  = os.path.join(BASE_DIR, "data", "raw")
MOL_DIR  = os.path.join(RAW_DIR, "molinetes")

print("MOL_DIR:", MOL_DIR, "exists?", os.path.exists(MOL_DIR))

# ---- Detect CSVs ----
mol_csvs = sorted(glob.glob(os.path.join(MOL_DIR, "*.csv")))
print("CSV detected:", len(mol_csvs))
for p in mol_csvs[:5]:
    print(" -", os.path.basename(p))
assert len(mol_csvs) > 0, "No CSV files found in data/raw/molinetes/."

# ---- Encoding & separator profile ----
ENC_LIST = ["utf-8-sig", "utf-8", "latin1", "cp1252"]
SEP = ";"   # confirmed by Week 1 debug

# ---- Helpers to ensure unique/clean column names ----
from collections import defaultdict

def normalize_token(s: str) -> str:
    s = (s or "").strip().strip('"').strip()
    s = re.sub(r"\s+", "_", s)
    s = s.lower()
    return s or "col"

def make_unique(cols):
    seen = defaultdict(int)
    out = []
    for c in cols:
        base = normalize_token(c)
        seen[base] += 1
        out.append(base if seen[base] == 1 else f"{base}_{seen[base]-1}")
    return out

# ---- Robust header reader (tries multiple encodings) ----
def read_header_names(path, enc_list=ENC_LIST, sep=SEP):
    last_err = None
    for enc in enc_list:
        try:
            with open(path, "r", encoding=enc, errors="strict") as f:
                first_line = f.readline().rstrip("\n\r")
            if first_line.startswith('"') and first_line.endswith('"'):
                first_line = first_line[1:-1]
            raw_cols = [c for c in first_line.split(sep)]
            return make_unique(raw_cols)
        except Exception as e:
            last_err = e
            continue
    # Fallback safe (replace invalid chars)
    with open(path, "r", encoding="utf-8", errors="replace") as f:
        first_line = f.readline().rstrip("\n\r")
    if first_line.startswith('"') and first_line.endswith('"'):
        first_line = first_line[1:-1]
    raw_cols = [c for c in first_line.split(sep)]
    return make_unique(raw_cols)

# ---- Robust CSV reader for quoted-rows + multiple encodings ----
def read_molinetes_quoted(path, enc_list=ENC_LIST, sep=SEP):
    cols = read_header_names(path, enc_list=enc_list, sep=sep)
    last_err = None
    for enc in enc_list:
        try:
            df = pd.read_csv(
                path,
                sep=sep,
                encoding=enc,
                engine="python",
                header=None,           # header ensuciado: lo pasamos manual en 'names'
                names=cols,
                quoting=csv.QUOTE_NONE, # ignorar quoting global
                on_bad_lines="skip"
            )
            # limpiar comillas/espacios residuales en columnas string
            for c in df.select_dtypes(include="object").columns:
                df[c] = df[c].astype(str).str.strip('"').str.strip()
            return df
        except Exception as e:
            last_err = e
            continue
    raise RuntimeError(f"Could not read {os.path.basename(path)} with encodings {enc_list}. Last error: {last_err}")

# ---- Load ALL CSVs, concat, and clean ----
df_list, failed = [], []
for p in mol_csvs:
    try:
        df = read_molinetes_quoted(p, enc_list=ENC_LIST, sep=SEP)
        df["source_file"] = os.path.basename(p)
        df_list.append(df)
    except Exception as e:
        failed.append((os.path.basename(p), repr(e)))

print(f"OK files: {len(df_list)} | Failed: {len(failed)}")
if failed:
    print("Failed examples:", failed[:3])

assert df_list, "No CSV could be loaded — check 'failed' list above."
mol_full = pd.concat(df_list, ignore_index=True)

# Drop spurious columns (if present)
for bad in ["col", "col_1", "unnamed: 1", "unnamed: 2"]:
    if bad in mol_full.columns:
        mol_full = mol_full.drop(columns=[bad], errors="ignore")

# Canonical rename
mol_full.columns = [c.strip().lower() for c in mol_full.columns]
rename_map = {}
for c in mol_full.columns:
    if c in {"fecha"}: rename_map[c] = "date"
    if c in {"desde", "desde_hora", "hora_desde"}: rename_map[c] = "time_from"
    if c in {"hasta", "hasta_hora", "hora_hasta"}: rename_map[c] = "time_to"
    if c in {"linea", "línea", "line"}: rename_map[c] = "line"
    if c in {"estacion", "estación", "station"}: rename_map[c] = "station"
    if c in {"pax_total","viajes","pasajeros","pax","passengers","conteo","count"}:
        rename_map[c] = "passengers"
mol_full = mol_full.rename(columns=rename_map)

# Remove header-like rows that slipped as data
mask_header_row = (
    mol_full.get("time_from", "").astype(str).str.upper().eq("DESDE") |
    mol_full.get("time_to", "").astype(str).str.upper().eq("HASTA")
)
mol_full = mol_full.loc[~mask_header_row].copy()

# Types and normalization
mol_full["date"] = pd.to_datetime(mol_full.get("date"), errors="coerce", dayfirst=True)
mol_full["year_month"] = mol_full["date"].dt.to_period("M").astype(str)

for col in ["time_from", "time_to", "station"]:
    if col in mol_full.columns:
        mol_full[col] = mol_full[col].astype(str).str.strip().str.upper()

# Passengers numeric (prefer pax_total if still present)
if "passengers" not in mol_full.columns and "pax_total" in mol_full.columns:
    mol_full["passengers"] = pd.to_numeric(mol_full["pax_total"], errors="coerce")
else:
    mol_full["passengers"] = pd.to_numeric(mol_full.get("passengers"), errors="coerce")

# Normalize line: "LineaB" / "LINEA B" -> "B"
if "line" in mol_full.columns:
    mol_full["line"] = (mol_full["line"]
                        .str.upper()
                        .str.replace(r"^LINEA\s*", "", regex=True)
                        .str.strip())

print("Shape:", mol_full.shape)
mol_full.head()


MOL_DIR: c:\Users\do_ch\OneDrive\Escritorio\Proyectos\Proyectos GitHub\subte-dashboard\data\raw\molinetes exists? True
CSV detected: 24
 - 202401_PAX15min-ABC.csv
 - 202401_PAX15min-DEH.csv
 - 202402_PAX15min-ABC.csv
 - 202402_PAX15min-DEH.csv
 - 202403_PAX15min-ABC.csv
OK files: 24 | Failed: 0
Shape: (11440440, 12)


,date,time_from,time_to,line,molinete,station,pax_pagos,pax_pases_pagos,pax_franq,passengers,source_file,year_month
1,2024-01-01,07:45:00,08:00:00,B,LineaB_Malabia_N_Turn01,MALABIA,3,0,0,3,202401_PAX15min-ABC.csv,2024-01
2,2024-01-01,07:45:00,08:00:00,B,LineaB_Tronador_Turn01,TRONADOR,1,0,0,1,202401_PAX15min-ABC.csv,2024-01
3,2024-01-01,07:45:00,08:00:00,B,LineaB_Pellegrini_E_Turn05,CARLOS PELLEGRINI,13,0,0,13,202401_PAX15min-ABC.csv,2024-01
4,2024-01-01,07:45:00,08:00:00,A,LineaA_Flores_Este_Turn03,FLORES,2,0,0,2,202401_PAX15min-ABC.csv,2024-01
5,2024-01-01,07:45:00,08:00:00,B,LineaB_Dorrego_N_Turn03,DORREGO,1,0,0,1,202401_PAX15min-ABC.csv,2024-01


In [4]:
# === Week 2: QA & sanity checks on mol_full ===
import numpy as np
import pandas as pd

assert "mol_full" in globals(), "mol_full not found. Run the load/clean block first."

print("Shape:", mol_full.shape)

# 1) Rango temporal y cobertura
date_min, date_max = mol_full["date"].min(), mol_full["date"].max()
print("Date range:", date_min, "→", date_max)

if "year_month" in mol_full.columns:
    months = mol_full["year_month"].dropna().unique()
    print("Year-Month coverage:", len(months), "months →", sorted(months)[:5], "...", sorted(months)[-5:])

# 2) Ratios de nulos en campos clave
keys = [c for c in ["date","line","station","passengers","time_from","time_to"] if c in mol_full.columns]
null_ratios = mol_full[keys].isna().mean().rename(lambda x: f"{x}_null_ratio").to_frame("ratio")
display(null_ratios.style.format("{:.2%}"))

# 3) Valores sospechosos
neg_pass = (mol_full["passengers"] < 0).sum() if "passengers" in mol_full.columns else 0
print("Negative passengers rows:", int(neg_pass))

zero_pass = (mol_full["passengers"] == 0).sum() if "passengers" in mol_full.columns else 0
print("Zero passengers rows:", int(zero_pass))

# 4) Duplicados potenciales (clave mínima por 15-min)
dup_cols = [c for c in ["date","time_from","time_to","line","station","molinete"] if c in mol_full.columns]
if dup_cols:
    dups = mol_full.duplicated(subset=dup_cols, keep=False).sum()
    print(f"Potential duplicates (subset={dup_cols}):", int(dups))
else:
    print("Duplicate check skipped (subset columns not all present).")

# 5) Resumen de archivos fuente (esperado: 24)
by_file = mol_full["source_file"].value_counts().rename_axis("source_file").to_frame("rows")
display(by_file.head(10))
print("Files count:", by_file.shape[0])

# 6) Distribución por línea y estación (muestras)
if "line" in mol_full.columns and "passengers" in mol_full.columns:
    agg_line_preview = (mol_full
                        .dropna(subset=["line","passengers"])
                        .groupby("line", as_index=False)["passengers"].sum()
                        .sort_values("passengers", ascending=False))
    display(agg_line_preview.head())

if "station" in mol_full.columns and "passengers" in mol_full.columns:
    agg_station_preview = (mol_full
                           .dropna(subset=["station","passengers"])
                           .groupby("station", as_index=False)["passengers"].sum()
                           .sort_values("passengers", ascending=False))
    display(agg_station_preview.head())

# 7) Quick stats de passengers
if "passengers" in mol_full.columns:
    desc = mol_full["passengers"].describe(percentiles=[0.5,0.9,0.99]).to_frame("passengers_stats")
    display(desc)

# 8) Muestras de filas problemáticas (si las hay)
issues = {}
if "passengers" in mol_full.columns:
    issues["negatives"] = mol_full.loc[mol_full["passengers"] < 0].head(3)
    issues["null_core"] = mol_full.loc[mol_full[["date","line","station","passengers"]].isna().any(axis=1)].head(3)

for k, df in issues.items():
    if not df.empty:
        print(f"\nSample problematic rows: {k}")
        display(df)


Shape: (11440440, 12)
Date range: 2024-01-01 00:00:00 → 2024-12-31 00:00:00
Year-Month coverage: 12 months → ['2024-01', '2024-02', '2024-03', '2024-04', '2024-05'] ... ['2024-08', '2024-09', '2024-10', '2024-11', '2024-12']


,ratio
date_null_ratio,0.00%
line_null_ratio,0.00%
station_null_ratio,0.00%
passengers_null_ratio,0.00%
time_from_null_ratio,0.00%
time_to_null_ratio,0.00%


Negative passengers rows: 0
Zero passengers rows: 44824
Potential duplicates (subset=['date', 'time_from', 'time_to', 'line', 'station', 'molinete']): 6


,rows
source_file,
202403_PAX15min-ABC.csv,599344
202408_PAX15min-ABC.csv,597421
202401_PAX15min-ABC.csv,587990
202404_PAX15min-ABC.csv,574669
202405_PAX15min-ABC.csv,569524
202407_PAX15min-ABC.csv,567561
202402_PAX15min-ABC.csv,551054
202406_PAX15min-ABC.csv,549191
202409_PAX15min-ABC.csv,540374


Files count: 24


,line,passengers
1,B,50423199
0,A,41170071
3,D,34606264
2,C,32528567
5,H,22606254


,station,passengers
22,CONSTITUCION,17666095
78,RETIRO,7018302
87,SAN PEDRITO,6511114
83,ROSAS,5763678
33,FEDERICO LACROZE,5734713


,passengers_stats
count,"11,440,440"
mean,18
std,21
min,0
50%,11
90%,42
99%,98
max,398


In [5]:
# === Week 2: Aggregates ===

# Total pasajeros por línea en todo 2024
agg_line_full = (
    mol_full
    .dropna(subset=["line","passengers"])
    .groupby("line", as_index=False)["passengers"]
    .sum()
    .sort_values("passengers", ascending=False)
)
display(agg_line_full)

# Tendencia mensual por línea
trend_full = (
    mol_full
    .dropna(subset=["year_month","line","passengers"])
    .groupby(["year_month","line"], as_index=False)["passengers"]
    .sum()
    .sort_values(["year_month","line"])
)
display(trend_full.head(18))  # primeras 18 filas para inspección


,line,passengers
1,B,50423199
0,A,41170071
3,D,34606264
2,C,32528567
5,H,22606254
4,E,20052333


,year_month,line,passengers
0,2024-01,A,3263982
1,2024-01,B,4239766
2,2024-01,C,2776085
3,2024-01,D,605572
4,2024-01,E,1411591
...,...,...,...
13,2024-03,B,4895806
14,2024-03,C,3092070
15,2024-03,D,3012727
16,2024-03,E,1934438


In [6]:
# === Block 6 (Lean): Visualizations + Export ===
import os
import plotly.express as px

# Si abriste el notebook “en frío”, recalcular agregados rápido:
if "agg_line_full" not in globals() or "trend_full" not in globals():
    agg_line_full = (
        mol_full.groupby("line", as_index=False)["passengers"]
        .sum()
        .sort_values("passengers", ascending=False)
    )
    trend_full = (
        mol_full.groupby(["year_month", "line"], as_index=False)["passengers"]
        .sum()
        .sort_values(["year_month", "line"])
    )

# 1) Barras: total por línea
fig_bar = px.bar(
    agg_line_full,
    x="line", y="passengers", text="passengers",
    title="Total Passengers by Subway Line (2024)",
    labels={"line": "Line", "passengers": "Passengers"},
)
fig_bar.update_traces(texttemplate='%{text:,.0f}', textposition='outside')
fig_bar.update_layout(uniformtext_minsize=8, uniformtext_mode='hide')
fig_bar.show()

# 2) Líneas: tendencia mensual por línea
fig_line = px.line(
    trend_full,
    x="year_month", y="passengers", color="line", markers=True,
    title="Monthly Demand by Line (2024)",
    labels={"year_month": "Year-Month", "passengers": "Passengers"},
)
fig_line.update_traces(mode="lines+markers")
fig_line.update_layout(xaxis_tickangle=-45)
fig_line.show()

# 3) Export (Kaleido)
bar_path  = os.path.join(PROC_DIR, "bar_passengers_by_line.png")
line_path = os.path.join(PROC_DIR, "line_trend_by_line.png")
fig_bar.write_image(bar_path,  scale=2)
fig_line.write_image(line_path, scale=2)
print("Charts saved in:", PROC_DIR)


Charts saved in: c:\Users\do_ch\OneDrive\Escritorio\Proyectos\Proyectos GitHub\subte-dashboard\data\processed


📌 Note / Nota

EN:
Below is a self-contained version of Block 6.
It re-loads and re-processes all raw CSVs (~11.4M rows), so it takes much longer to run.
👉 You don’t need to execute it to reproduce the results shown in the screenshot or the exported PNGs (bar_passengers_by_line.png, line_trend_by_line.png).
Use this block only if you want to run the notebook independently from scratch without relying on previous blocks.

ES:
Aquí debajo está la versión autocontenida del Bloque 6.
Esta versión vuelve a cargar y procesar todos los CSV crudos (~11,4M de filas), por lo que tarda mucho más en ejecutarse.
👉 No es necesario ejecutarla para ver los resultados mostrados en la captura o en los PNG exportados (bar_passengers_by_line.png, line_trend_by_line.png).
Utilízala solo si deseas ejecutar el notebook de manera independiente desde cero, sin depender de los bloques previos.

In [ ]:
# === Block 6 (Self-contained): Load + Normalize + Aggregate + Visualize + Export ===
import os, re, csv, glob
import pandas as pd
import plotly.express as px

# Paths
BASE_DIR = os.path.abspath("..")
RAW_DIR  = os.path.join(BASE_DIR, "data", "raw")
PROC_DIR = os.path.join(BASE_DIR, "data", "processed")
MOL_DIR  = os.path.join(RAW_DIR, "molinetes")
os.makedirs(PROC_DIR, exist_ok=True)

# Config
SEP = ";"
ENC_LIST = ["utf-8-sig", "utf-8", "latin1", "cp1252"]

def normalize_token(s: str) -> str:
    s = (s or "").strip().strip('"').strip()
    s = re.sub(r"\s+", "_", s)
    return s.lower() or "col"

def make_unique(cols):
    seen = {}
    out = []
    for c in cols:
        base = normalize_token(c)
        seen[base] = seen.get(base, 0) + 1
        out.append(base if seen[base] == 1 else f"{base}_{seen[base]-1}")
    return out

def read_header_names(path, enc_list=ENC_LIST, sep=SEP):
    for enc in enc_list:
        try:
            with open(path, "r", encoding=enc, errors="strict") as f:
                first_line = f.readline().rstrip("\n\r")
            if first_line.startswith('"') and first_line.endswith('"'):
                first_line = first_line[1:-1]
            return make_unique(first_line.split(sep))
        except Exception:
            continue
    with open(path, "r", encoding="utf-8", errors="replace") as f:
        first_line = f.readline().rstrip("\n\r")
    if first_line.startswith('"') and first_line.endswith('"'):
        first_line = first_line[1:-1]
    return make_unique(first_line.split(sep))

def read_molinetes_quoted(path, enc_list=ENC_LIST, sep=SEP):
    cols = read_header_names(path, enc_list=enc_list, sep=sep)
    for enc in enc_list:
        try:
            df = pd.read_csv(
                path, sep=sep, encoding=enc, engine="python",
                header=None, names=cols, quoting=csv.QUOTE_NONE, on_bad_lines="skip",
            )
            for c in df.select_dtypes(include="object").columns:
                df[c] = df[c].astype(str).str.strip('"').str.strip()
            return df
        except Exception:
            continue
    # Fallback final
    df = pd.read_csv(
        path, sep=sep, encoding="utf-8", encoding_errors="replace",
        engine="python", header=None, names=cols, quoting=csv.QUOTE_NONE, on_bad_lines="skip",
    )
    for c in df.select_dtypes(include="object").columns:
        df[c] = df[c].astype(str).str.strip('"').str.strip()
    return df

# 1) Load all CSVs
csvs = sorted(glob.glob(os.path.join(MOL_DIR, "*.csv")))
assert csvs, f"No se hallaron CSV en {MOL_DIR}"
df_list = []
for p in csvs:
    df = read_molinetes_quoted(p)
    df["source_file"] = os.path.basename(p)
    df_list.append(df)
mol_full = pd.concat(df_list, ignore_index=True)
mol_full.columns = [c.strip().lower() for c in mol_full.columns]

# 2) Normalize fields
if "fecha" in mol_full.columns:
    mol_full["date"] = pd.to_datetime(mol_full["fecha"], errors="coerce", dayfirst=True)
else:
    mol_full["date"] = pd.to_datetime(mol_full.get("date"), errors="coerce", dayfirst=True)
mol_full["year_month"] = mol_full["date"].dt.to_period("M").astype(str)

mol_full["line"] = mol_full.get("linea", mol_full.get("line", "")).astype(str)
mol_full["line"] = (mol_full["line"].str.upper()
                    .str.replace(r"^LINEA\s*", "", regex=True)
                    .str.strip())
mol_full["station"] = mol_full.get("estacion", mol_full.get("station", "")).astype(str).str.upper().str.strip()

if "pax_total" in mol_full.columns:
    mol_full["passengers"] = pd.to_numeric(mol_full["pax_total"], errors="coerce")
elif "passengers" in mol_full.columns:
    mol_full["passengers"] = pd.to_numeric(mol_full["passengers"], errors="coerce")
else:
    parts = [c for c in ["pax_pagos","pax_pases_pagos","pax_franq"] if c in mol_full.columns]
    if parts:
        mol_full["passengers"] = pd.to_numeric(mol_full[parts].sum(axis=1), errors="coerce")
    else:
        raise RuntimeError("No encontré columna de pasajeros.")

# 3) Aggregates
agg_line_full = (
    mol_full.groupby("line", as_index=False)["passengers"]
    .sum()
    .sort_values("passengers", ascending=False)
)
trend_full = (
    mol_full.groupby(["year_month", "line"], as_index=False)["passengers"]
    .sum()
    .sort_values(["year_month", "line"])
)

# 4) Visuals
fig_bar = px.bar(
    agg_line_full, x="line", y="passengers", text="passengers",
    title="Total Passengers by Subway Line (2024)",
    labels={"line": "Line", "passengers": "Passengers"},
)
fig_bar.update_traces(texttemplate='%{text:,.0f}', textposition='outside')
fig_bar.update_layout(uniformtext_minsize=8, uniformtext_mode='hide')
fig_bar.show()

fig_line = px.line(
    trend_full, x="year_month", y="passengers", color="line", markers=True,
    title="Monthly Demand by Line (2024)",
    labels={"year_month": "Year-Month", "passengers": "Passengers"},
)
fig_line.update_traces(mode="lines+markers")
fig_line.update_layout(xaxis_tickangle=-45)
fig_line.show()

# 5) Export
bar_path  = os.path.join(PROC_DIR, "bar_passengers_by_line.png")
line_path = os.path.join(PROC_DIR, "line_trend_by_line.png")
fig_bar.write_image(bar_path, scale=2)
fig_line.write_image(line_path, scale=2)
print("Charts saved:", bar_path, "and", line_path)
